In [11]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-06-30 16:38:44--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-30 16:38:44 (18.3 MB/s) - ‘minsearch.py’ saved [4073/4073]



In [12]:
import requests 
import json
import minsearch
from mistralai import Mistral

docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
if docs_response.status_code == 200:
    documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

/workspaces/llm-zoomcamp/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [4]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [14]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    index = minsearch.Index(
        text_fields=["question", "text", "section"],
        keyword_fields=["course"]
    )
    index.fit(documents)

    result = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=15
    )
    return result

In [15]:
def build_prompt(query, search_result):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT dosen't contain the answer, output None.
        
        QUESTION: {question}
        
        CONTEXT: 
        {context}
    """.strip()

    context = ""

    for doc in search_result:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [23]:
def llm(prompt):
    mistral_client = Mistral(api_key="4JwtBsMx2DGAj0kfL2JpVn3j4f3vCkSI")
    response = mistral_client.chat.complete(
    model="mistral-large-latest",
        messages=[{
            "role": "user",
            "content": prompt
    }]
    )
    return response.choices[0].message.content

In [24]:
def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result=search_result)
    result = llm(prompt)
    return result 

In [26]:
rag("how do i run kafka?")

'To run Kafka, you can use the following steps based on the context provided:\n\n1. **For Java Kafka**:\n   - In the project directory, run the following command to execute a Java Kafka producer:\n     ```sh\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n2. **For Python Kafka**:\n   - Create a virtual environment and install the necessary dependencies:\n     ```sh\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n   - Ensure that Docker images are up and running before executing your Python scripts.\n\nIf the context does not provide additional details on running Kafka in other scenarios, please refer to the specific FAQ entries for more information.'

In [27]:
from qdrant_client import QdrantClient, models

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
qd_client = QdrantClient("http://localhost:6333")

In [34]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [35]:
collection_name = "zoomcamp-faq"

if client.collection_exists(collection_name):
    print(f"collection {collection_name} already exists")
else:    
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=EMBEDDING_DIMENSIONALITY,
            distance=models.Distance.COSINE
        )
    )

In [36]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [37]:
points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

In [38]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [39]:
question = 'I just discovered the course. Can I still join it?'

In [41]:
def vector_search(question):
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    results = []
    for point in query_points.points:
        results.append(point.payload)
    return results

In [42]:
vector_search('How do i run kafka?')

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'For example, when running JsonConsumer.java, got:\nConsuming form kafka started\nRESULTS:::0\nRESULTS:::0\nRESULTS:::0\nOr when running JsonProducer.java, got:\nException in thread "main" java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.SaslAuthenticationException: Authentication failed\nSolution:\nMake sure in the scripts in src/main/java/org/example/ that you are running (e.g. JsonConsumer.java, JsonProducer.java), the StreamsConfig.BOOTSTRAP_SERVERS_CONFIG is the correct server url (e.g. europe-west3 from example vs europe-west2)\nMake sure cluster key and secrets are updated in src/main/java/org/example/Secrets.java (KAFKA_CLUSTER_KEY and KA

In [43]:
def rag(query):
    search_result = vector_search(query)
    prompt = build_prompt(query, search_result=search_result)
    result = llm(prompt)
    return result 

In [44]:
rag('How do I run kafka?')

'To run Kafka in the context of the provided information, you would need to ensure that your Kafka broker Docker container is running. If you encounter the error `kafka.errors.NoBrokersAvailable: NoBrokersAvailable`, it likely means that your Kafka broker Docker container is not working. To resolve this, you can use the following steps:\n\n1. Use `docker ps` to confirm if the Kafka broker Docker container is running.\n2. If it is not running, navigate to the directory containing your Docker Compose YAML file and run `docker compose up -d` to start all the instances.\n\nFor running specific Java scripts related to Kafka (e.g., producer, consumer), you can use the following command in the project directory:\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'

In [45]:
rag('the course is already started. Can I still enroll?')

"Yes, you can still enroll. Even if you don't register, you're still eligible to submit the homeworks."